In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
from pprint import pprint
from datetime import datetime, timedelta

In [2]:
def read_hotspot_terra_streaming_data():
    hotspot_terra_streaming_lst=[]
    with open('hotspot_TERRA_streaming.csv') as read_file:
        read_file.readline() # for skipping header
        acqua=read_file.readlines()
        for a in acqua:
             # create dummy doc
            tmp_doc={
            "latitude":0.0,
            "date":"2021-12-27",
            "time":"04:16:51",
            "longitude":0.0,
            "confidence":0,
            "surface_temperature_celcius":0,
            "key":"terra"
            }
            # update tmp_doc
            lst=a.split(',')
            tmp_doc["latitude"]=float(lst[0])
            tmp_doc["longitude"]=float(lst[1])
            tmp_doc["confidence"]=int(lst[2])
            tmp_doc["surface_temperature_celcius"]=int(lst[3].split('\n')[0])
            
            # update hotspot_terra_streaming_lst
            hotspot_terra_streaming_lst.append(tmp_doc)
    return hotspot_terra_streaming_lst

In [3]:
print(len(read_hotspot_terra_streaming_data()))

2623


In [4]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [5]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [6]:
if __name__ == '__main__':
   
    topic = 'Assgn'
    
    # retrieving data from csv file
    hotspot_terra_streaming_lst=read_hotspot_terra_streaming_data()
    
    
    # check size of hotspot lst
    size= len(hotspot_terra_streaming_lst)
    
    print('Publishing records..')
    producer_terra = connect_kafka_producer()
    
 
    # set random_index
    random_index=0
    #set datetime_val
    datetime_val=datetime.strptime(hotspot_terra_streaming_lst[random_index]['date'],'%Y-%m-%d') 
    
     # create a random list of index ranging from 0 to 2643 to help us randomly access a different
    # terra data at each iteration
    random_lst=random.sample(range(0, 2623), 2623)
    
    for e in range(2623):
        
        # update hotspot_terra_streaming_lst's datetime attribute 
        # at random_index pos (if every 10 secs is equivalent to
        # 24 hours then 2 secs is equivalent to 4.8 hrs)
        if random_index>0:    
            datetime_val=datetime_val + timedelta(hours=4.8)
        
        # assign time to 'time' attribute of hotspot_aqua_streaming_lst
        t_time=datetime.strptime(str(datetime_val), '%Y-%m-%d %H:%M:%S')
        val="{:02d}:{:02d}:{:02d}".format(t_time.hour,t_time.minute, t_time.second)
        
        print(val)
        
        # assign 'time' value to the datetime attr at random index of lst
        hotspot_terra_streaming_lst[random_lst[random_index]]['time']=str(val)
        
        
        # assign the updated position record to data
        data=hotspot_terra_streaming_lst[random_lst[random_index]]
        
        
        # publish message
        publish_message(producer_terra, topic, 'terra', dumps(data))
        
        # incremet random index
        random_index+=1
        
        # sleep of 2 second to simulate real-time streaming data
        sleep(2)

Publishing records..
00:00:00
Message published successfully. Data: {"latitude": -36.0574, "date": "2021-12-27", "time": "00:00:00", "longitude": 141.5856, "confidence": 84, "surface_temperature_celcius": 57, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -36.3823, "date": "2021-12-27", "time": "04:48:00", "longitude": 141.3146, "confidence": 95, "surface_temperature_celcius": 77, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -37.3851, "date": "2021-12-27", "time": "09:36:00", "longitude": 145.8878, "confidence": 83, "surface_temperature_celcius": 63, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -37.4022, "date": "2021-12-27", "time": "14:24:00", "longitude": 143.2016, "confidence": 69, "surface_temperature_celcius": 45, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -36.3831, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.7276, "confidence": 71, "surface_t

04:48:00
Message published successfully. Data: {"latitude": -35.4993, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.3105, "confidence": 84, "surface_temperature_celcius": 57, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.3477, "date": "2021-12-27", "time": "09:36:00", "longitude": 144.533, "confidence": 86, "surface_temperature_celcius": 41, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -37.4854, "date": "2021-12-27", "time": "14:24:00", "longitude": 145.7008, "confidence": 64, "surface_temperature_celcius": 44, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -36.6158, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.4424, "confidence": 100, "surface_temperature_celcius": 101, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -36.3712, "date": "2021-12-27", "time": "00:00:00", "longitude": 144.5011, "confidence": 87, "surface_temperature_celcius":

09:36:00
Message published successfully. Data: {"latitude": -36.663, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.3038, "confidence": 93, "surface_temperature_celcius": 72, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -36.9654, "date": "2021-12-27", "time": "14:24:00", "longitude": 140.981, "confidence": 69, "surface_temperature_celcius": 44, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -36.2938, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.6237, "confidence": 63, "surface_temperature_celcius": 42, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -35.723, "date": "2021-12-27", "time": "00:00:00", "longitude": 142.063, "confidence": 64, "surface_temperature_celcius": 54, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -38.1748, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.0552, "confidence": 69, "surface_temperature_celcius": 44, 

14:24:00
Message published successfully. Data: {"latitude": -36.9524, "date": "2021-12-27", "time": "14:24:00", "longitude": 143.7501, "confidence": 60, "surface_temperature_celcius": 40, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -37.1719, "date": "2021-12-27", "time": "19:12:00", "longitude": 141.5775, "confidence": 71, "surface_temperature_celcius": 46, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -37.0899, "date": "2021-12-27", "time": "00:00:00", "longitude": 141.0238, "confidence": 100, "surface_temperature_celcius": 90, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -37.636, "date": "2021-12-27", "time": "04:48:00", "longitude": 149.33, "confidence": 94, "surface_temperature_celcius": 43, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.7842, "date": "2021-12-27", "time": "09:36:00", "longitude": 146.9658, "confidence": 53, "surface_temperature_celcius": 39

19:12:00
Message published successfully. Data: {"latitude": -36.4392, "date": "2021-12-27", "time": "19:12:00", "longitude": 144.2111, "confidence": 69, "surface_temperature_celcius": 44, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -37.0753, "date": "2021-12-27", "time": "00:00:00", "longitude": 141.0532, "confidence": 79, "surface_temperature_celcius": 52, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -36.285, "date": "2021-12-27", "time": "04:48:00", "longitude": 146.151, "confidence": 81, "surface_temperature_celcius": 54, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.9285, "date": "2021-12-27", "time": "09:36:00", "longitude": 143.9622, "confidence": 66, "surface_temperature_celcius": 43, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -36.9221, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.9429, "confidence": 79, "surface_temperature_celcius": 52

...

14:24:00
Message published successfully. Data: {"latitude": -37.608, "date": "2021-12-27", "time": "14:24:00", "longitude": 149.292, "confidence": 100, "surface_temperature_celcius": 55, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -36.1748, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.7582, "confidence": 91, "surface_temperature_celcius": 69, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -37.9112, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.6025, "confidence": 85, "surface_temperature_celcius": 61, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -38.0524, "date": "2021-12-27", "time": "04:48:00", "longitude": 141.6738, "confidence": 76, "surface_temperature_celcius": 49, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.9664, "date": "2021-12-27", "time": "09:36:00", "longitude": 142.1561, "confidence": 81, "surface_temperature_celcius": 5

19:12:00
Message published successfully. Data: {"latitude": -37.9526, "date": "2021-12-27", "time": "19:12:00", "longitude": 143.597, "confidence": 100, "surface_temperature_celcius": 107, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -36.3492, "date": "2021-12-27", "time": "00:00:00", "longitude": 145.9333, "confidence": 73, "surface_temperature_celcius": 47, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -36.2646, "date": "2021-12-27", "time": "04:48:00", "longitude": 141.0021, "confidence": 82, "surface_temperature_celcius": 55, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.2391, "date": "2021-12-27", "time": "09:36:00", "longitude": 142.2226, "confidence": 76, "surface_temperature_celcius": 49, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -36.0963, "date": "2021-12-27", "time": "14:24:00", "longitude": 145.6162, "confidence": 68, "surface_temperature_celcius":

00:00:00
Message published successfully. Data: {"latitude": -37.5012, "date": "2021-12-27", "time": "00:00:00", "longitude": 142.5853, "confidence": 100, "surface_temperature_celcius": 90, "key": "terra"}
04:48:00
Message published successfully. Data: {"latitude": -36.5985, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.1836, "confidence": 100, "surface_temperature_celcius": 103, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.664, "date": "2021-12-27", "time": "09:36:00", "longitude": 142.8802, "confidence": 100, "surface_temperature_celcius": 87, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -35.9124, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.8639, "confidence": 86, "surface_temperature_celcius": 61, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -36.0799, "date": "2021-12-27", "time": "19:12:00", "longitude": 146.0784, "confidence": 77, "surface_temperature_celcius

04:48:00
Message published successfully. Data: {"latitude": -36.6463, "date": "2021-12-27", "time": "04:48:00", "longitude": 144.7594, "confidence": 59, "surface_temperature_celcius": 41, "key": "terra"}
09:36:00
Message published successfully. Data: {"latitude": -36.3086, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.135, "confidence": 82, "surface_temperature_celcius": 55, "key": "terra"}
14:24:00
Message published successfully. Data: {"latitude": -38.0792, "date": "2021-12-27", "time": "14:24:00", "longitude": 143.7361, "confidence": 94, "surface_temperature_celcius": 93, "key": "terra"}
19:12:00
Message published successfully. Data: {"latitude": -37.633, "date": "2021-12-27", "time": "19:12:00", "longitude": 149.264, "confidence": 64, "surface_temperature_celcius": 42, "key": "terra"}
00:00:00
Message published successfully. Data: {"latitude": -37.7935, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.1606, "confidence": 96, "surface_temperature_celcius": 78,

KeyboardInterrupt: 

In [ ]:
data=read_hotspot_terra_streaming_data()
print(data[0])